#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation for Adult race.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [13]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle

#### Load dataset and specify options


Huangrui's ADULT PREPROCESS

Only modify the sex and race to 1/0

With favorable one as 1. 

keep all other features

Race Is not dummy columned, thus we have fewer columns


# Huangrui defined Adult Dataset

In [14]:
default_mappings = {
    'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
    'protected_attribute_maps': [{1.0: 'Non-Black', 0.0: 'Black'},
                                 {1.0: 'Male', 0.0: 'Female'}]
}
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
class AdultDataset(StandardDataset):
    """Adult Census Income Dataset.

    See :file:`aif360/data/raw/adult/README.md`.
    """

    def __init__(self, label_name='Y',
                 favorable_classes=[1],
                 protected_attribute_names=["race_Black"],
                 privileged_classes=[ [0]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[], features_to_drop=[],
                 na_values=['?'], custom_preprocessing=None,
                 metadata=default_mappings, path=None):
         #deal with the race
         df = pd.read_csv("./Huangrui/adult/adult_train1.csv",index_col=False,header = 0, sep = ',')
         numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
         code_continuous(df,numericals, 5)
         df.drop(["race_Asian-Pac-Islander",
            "race_Amer-Indian-Eskimo",
            "race_White",
            "race_Other"], axis=1, inplace=True)
         super(AdultDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

In [15]:
## import dataset
dataset_used = "adult" # "adult", "german", "compas"

privileged_groups = [{'race_Black': 0}]
unprivileged_groups = [{'race_Black': 1}]

# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

### Train classifier on original data

In [18]:
experiments_info = {}
budget = 0.01
for K in range(1, 6):
    dataset_orig_train= AdultDataset(path="./Huangrui/adult/adult_train{}.csv".format(K),protected_attribute_names=['race_Black'],
                privileged_classes=[[0]])
    dataset_orig_test= AdultDataset(path="./Huangrui/adult/adult_test{}.csv".format(K),protected_attribute_names=['race_Black'],
                privileged_classes=[[0]])
    #only use the budget% of the training data
    dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)

    # Lasso linear classifier and predictions
    X_train = dataset_orig_train.features
    y_train = dataset_orig_train.labels.ravel()
    lmod = pickle.load(open('experiments/adult'+str(K)+'_race_bmodel.pkl','rb'))["clf"]
   
    y_train_pred = lmod.predict(X_train)

    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.labels = y_train_pred.reshape(-1,1)
    sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
    dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)

    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = dataset_orig_test_pred.features
    y_test = dataset_orig_test_pred.labels
    dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)

    #### Best threshold for classification only (no fairness)
    num_thresh = 100
    ba_arr = np.zeros(num_thresh)
    class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
    for idx, class_thresh in enumerate(class_thresh_arr):
        
        fav_inds = dataset_orig_train_pred.scores > class_thresh
        dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
        dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
        
        classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                                dataset_orig_train_pred, 
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        
        ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                        +classified_metric_orig_train.true_negative_rate())

    best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
    best_class_thresh = class_thresh_arr[best_ind]

    # print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
    # print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)
    #### Estimate optimal parameters for the ROC method
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    low_class_thresh=0.01, high_class_thresh=0.99,
                                    num_class_thresh=100, num_ROC_margin=50,
                                    metric_name=metric_name,
                                    metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)
    # print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    # print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the train set
    fav_inds = dataset_orig_train_pred.scores > best_class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

    # display(Markdown("#### train set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                    unprivileged_groups, privileged_groups)

    # Transform the validation set
    dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

    # display(Markdown("#### train set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                    unprivileged_groups, privileged_groups)
    # Testing: Check if the metric optimized has not become worse
    assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

    # Metrics for the test set
    fav_inds = dataset_orig_test_pred.scores > best_class_thresh
    dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
    dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                    unprivileged_groups, privileged_groups)
    # Metrics for the transformed test set
    dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                    unprivileged_groups, privileged_groups)

    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}

C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.8185
Statistical parity difference = -0.1107
Disparate impact = 0.6930
Average odds difference = 0.0100
Equal opportunity difference = 0.1731
Theil index = 0.0813
Balanced accuracy = 0.7499
Statistical parity difference = -0.1436
Disparate impact = 0.7581
Average odds difference = -0.0640
Equal opportunity difference = 0.0385
Theil index = 0.0679
Balanced accuracy = 0.7932
Statistical parity difference = -0.1767
Disparate impact = 0.4577
Average odds difference = -0.1011
Equal opportunity difference = -0.0994
Theil index = 0.0989
Balanced accuracy = 0.7666
Statistical parity difference = -0.2274
Disparate impact = 0.5964
Average odds difference = -0.1103
Equal opportunity difference = -0.0427
Theil index = 0.0710
K = 1, budget = 0.01
The Error for the test dataset is 0.3278
The Equal opportunity difference for the test dataset is -0.04268


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.8273
Statistical parity difference = -0.0377
Disparate impact = 0.8690
Average odds difference = 0.0778
Equal opportunity difference = 0.2308
Theil index = 0.0836
Balanced accuracy = 0.8189
Statistical parity difference = -0.0829
Disparate impact = 0.6441
Average odds difference = -0.0227
Equal opportunity difference = 0.0385
Theil index = 0.0903
Balanced accuracy = 0.7779
Statistical parity difference = -0.1578
Disparate impact = 0.4285
Average odds difference = -0.1085
Equal opportunity difference = -0.1371
Theil index = 0.1106
Balanced accuracy = 0.7551
Statistical parity difference = -0.1423
Disparate impact = 0.3796
Average odds difference = -0.1198
Equal opportunity difference = -0.1770
Theil index = 0.1250
K = 2, budget = 0.01
The Error for the test dataset is 0.1662
The Equal opportunity difference for the test dataset is -0.177


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.8300
Statistical parity difference = -0.0331
Disparate impact = 0.8831
Average odds difference = 0.0808
Equal opportunity difference = 0.2308
Theil index = 0.0825
Balanced accuracy = 0.7499
Statistical parity difference = -0.1436
Disparate impact = 0.7581
Average odds difference = -0.0640
Equal opportunity difference = 0.0385
Theil index = 0.0679
Balanced accuracy = 0.7776
Statistical parity difference = -0.1555
Disparate impact = 0.4330
Average odds difference = -0.1047
Equal opportunity difference = -0.1312
Theil index = 0.1109
Balanced accuracy = 0.7665
Statistical parity difference = -0.2229
Disparate impact = 0.6028
Average odds difference = -0.1084
Equal opportunity difference = -0.0442
Theil index = 0.0715
K = 3, budget = 0.01
The Error for the test dataset is 0.3269
The Equal opportunity difference for the test dataset is -0.04424


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.8333
Statistical parity difference = -0.0783
Disparate impact = 0.6570
Average odds difference = -0.0263
Equal opportunity difference = 0.0192
Theil index = 0.0831
Balanced accuracy = 0.8333
Statistical parity difference = -0.0783
Disparate impact = 0.6570
Average odds difference = -0.0263
Equal opportunity difference = 0.0192
Theil index = 0.0831
Balanced accuracy = 0.7552
Statistical parity difference = -0.1469
Disparate impact = 0.3528
Average odds difference = -0.1276
Equal opportunity difference = -0.1891
Theil index = 0.1250
Balanced accuracy = 0.7552
Statistical parity difference = -0.1469
Disparate impact = 0.3528
Average odds difference = -0.1276
Equal opportunity difference = -0.1891
Theil index = 0.1250
K = 4, budget = 0.01
The Error for the test dataset is 0.1646
The Equal opportunity difference for the test dataset is -0.1891


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.8273
Statistical parity difference = -0.0377
Disparate impact = 0.8690
Average odds difference = 0.0778
Equal opportunity difference = 0.2308
Theil index = 0.0836
Balanced accuracy = 0.7472
Statistical parity difference = -0.1482
Disparate impact = 0.7523
Average odds difference = -0.0670
Equal opportunity difference = 0.0385
Theil index = 0.0679
Balanced accuracy = 0.7772
Statistical parity difference = -0.1544
Disparate impact = 0.4293
Average odds difference = -0.1006
Equal opportunity difference = -0.1230
Theil index = 0.1113
Balanced accuracy = 0.7664
Statistical parity difference = -0.2242
Disparate impact = 0.6008
Average odds difference = -0.1076
Equal opportunity difference = -0.0407
Theil index = 0.0714
K = 5, budget = 0.01
The Error for the test dataset is 0.3271
The Equal opportunity difference for the test dataset is -0.04067


In [19]:
experiments_info

{'K = 1, budget = 0.01': {'Error': 0.32780532522188427,
  'Equal opportunity difference': -0.04268318204410493},
 'K = 2, budget = 0.01': {'Error': 0.1661735905662736,
  'Equal opportunity difference': -0.17699400946974653},
 'K = 3, budget = 0.01': {'Error': 0.32693028876203173,
  'Equal opportunity difference': -0.04424032751719986},
 'K = 4, budget = 0.01': {'Error': 0.1645901912579691,
  'Equal opportunity difference': -0.18908232401182407},
 'K = 5, budget = 0.01': {'Error': 0.32713863077628236,
  'Equal opportunity difference': -0.040668898945771303}}